In [1]:
# preamble
from datetime import datetime
start = datetime.now()

## Step 1 > import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [2]:
#import libraries
import pandas as pd
import numpy as np
import os

#define "root' path 
ICpath = r'C:\Users\joseh\Instacart Basket Analysis'

#import data set orders_products_derived_fields.pkl
ords_prods_merge = pd.read_pickle(os.path.join(ICpath, '02 Data', 'Prepared Data', 'orders_products_derived_fields.pkl'))
ords_prods_merge #verify what has been imported looks ok

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order_from_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_label,busiest_day,Busiest_Days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,Mid-range product,Regular busy,regular busy,Most orders
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Regular busy,slowest days,Average orders
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Regular busy,slowest days,Most orders
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,slowest days,Average orders
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,slowest days,Most orders
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,1320836,202557,17,2,15,1.0,False,43553,2,1,Orange Energy Shots,64,7,3.7,Low-range product,Regular busy,regular busy,Most orders
32404855,31526,202557,18,5,11,3.0,False,43553,2,1,Orange Energy Shots,64,7,3.7,Low-range product,Regular busy,regular busy,Most orders
32404856,758936,203436,1,2,7,NaN,True,42338,4,0,"Zucchini Chips, Pesto",50,19,6.9,Mid-range product,Regular busy,regular busy,Average orders
32404857,2745165,203436,2,3,5,15.0,False,42338,16,1,"Zucchini Chips, Pesto",50,19,6.9,Mid-range product,Regular busy,slowest days,Fewest orders


## Step 2 > In this Exercise, you learned how to find the aggregated mean of the ```order_number``` column grouped by ```department_id``` for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [3]:
ords_prods_merge.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## Step 3 > Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

### NOTE: for comparison purposes see the result for the first 1M rows in the next cell 

#### Observations
* The subset of the first 1 Million rows displays 8 ```department_id```s VS. the total of 21
* When we take into account the entire dataframe, order_number means is lower in 6 of 8 ```department_id```s (4,7,13,14,16,20)
* The means range from 11.29 to 19.46 in the small sample
* The means range from 15.22 to 22.90 in the full data set

In [4]:
#for comparison presenting the aggregation results from the first 1Million records
ords_prods_merge[:1000000].groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


## Step 4 > Follow the instructions in the Exercise for creating a ```loyalty flag``` for existing customers using the ```transform()``` and ```loc()``` functions.

In [5]:
# create a new column with the MAX order number for each user_id
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)
# inspect the results
ords_prods_merge['max_order'].value_counts()

99    1171333
8      811843
6      811396
9      810213
7      803979
       ...   
97      44949
98      44587
96      40453
2           6
1           5
Name: max_order, Length: 99, dtype: int64

In [6]:
# assigning a new label to express loyalty level as follows
# Loyalty customer -- 41 or more orders
# Regular customer -- from 11 to 40 orders
# New customer -- 10 orders or less

# assigning the new label in the column 'loyalty_flag' to

# 1) the subset of records with 41 or more orders
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag' ] = 'Loyal customer'

In [7]:
# 2) subset of records with 11 to 40 orders
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) &
                     (ords_prods_merge['max_order'] > 10), 'loyalty_flag' ] = 'Regular customer'

In [8]:
# 3) subset of records with 10 or less orders
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag' ] = 'New customer'

In [9]:
# checking the results
ords_prods_merge['loyalty_flag'].value_counts()

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [10]:
#verifying size+frequency of new columns
15876776 + 10284093 + 6243990

32404859

## Step 5 >	The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the ```product price```s for each ```loyalty``` category (```Loyal Customer```, ```Regular Customer```, and ```New Customer```). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [11]:
# using the aggregate function 'agg' to derive basic descriptive statistics
ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['min', 'median', 'mean', 'max']})

prices                           
                    min median       mean      max
loyalty_flag                                      
Loyal customer      1.0    7.4  10.386336  99999.0
New customer        1.0    7.4  13.294670  99999.0
Regular customer    1.0    7.4  12.495717  99999.0

### Further investigation (see following code cells) reveal the following:
+ The ```max``` price value of 99,999 is unusual and requires further investigation & questioning
+ From analysis of the dataset I determined typical prices do not exceed $25
+ All prices > 25 are either 14,900 or 99,999 --there are ```5,127``` records (or 0.016%)``` with unusually high prices
+ All ```4,429``` items priced at 14,900 are for the same product name: ```Lowfat 2% Milkfat Cottage Cheese```
+ All ``` 698``` items priced at 99,999 are for the same product name:  ```2 % Reduced Fat  Milk```
+ There are no entries for these two products with a different price


### Overview of customers by loyalty
+ The means for ```Regular-``` and ```New-customer```s is pulled higher thanks to the outliers/evidence presented
+ From the agregate resulst above, we could conclude that ```Loyal``` customers spend on average $3 less than the rest. However, when we eliminate the effect of outliers this difference loses significance (see comments later in this section)

In [12]:
# investigating I determined most prices are <= 25
# showing that items with prices >25 fall in 2 different prices: 14,900 and 99,999
ords_prods_merge[ords_prods_merge['prices']>25]['prices'].value_counts()

14900.0    4429
99999.0     698
Name: prices, dtype: int64

In [13]:
#this code shows the distribution of the price 14,900 among customer's loyalty category
ords_prods_merge[ords_prods_merge['prices']==14900.0]['loyalty_flag'].value_counts()

Regular customer    2235
Loyal customer      1247
New customer         947
Name: loyalty_flag, dtype: int64

In [14]:
#this code shows the distribution of price 99,999 among customer's loyalty categories
ords_prods_merge[ords_prods_merge['prices']==99999.0]['loyalty_flag'].value_counts()

Regular customer    413
New customer        202
Loyal customer       83
Name: loyalty_flag, dtype: int64

In [15]:
# this code shows ALL items priced at 14,900 refer to the same product: Lowfat 2% Milkfat Cottage Cheese
ords_prods_merge[ords_prods_merge['prices']==14900.0]['product_name'].value_counts()

Lowfat 2% Milkfat Cottage Cheese    4429
Name: product_name, dtype: int64

In [16]:
# this code shows ALL items priced at 99,999 refer to the same product: 2% reduced Fat Milk
ords_prods_merge[ords_prods_merge['prices']==99999.0]['product_name'].value_counts()

2 % Reduced Fat  Milk    698
Name: product_name, dtype: int64

### Redoing the aggregate calculation for loyal customers after removing the items with outlier ```prices```

+ in the following code cells I have removed the outlier-prices: 14,900.0 and 99,999.0
+ recalculating ```MIN/MEDIAN/MEAN/MAX``` under these circumstances changes the mean considerably
+ although ```Loyal``` customers have a lower ```mean``` price (7.773) it is not significantly less than the cases for Regular (7.798) and New (7.801) customers

### Conclusion: 
#### based on prices alone it is not possible to create a clear-cut segmentation between ```Loyal``` and other customers

In [17]:
# assign the subset of normal prices to a new data frame
# with the purpose of getting statistics without the outliers
# I determined the outliers represent 0.016% of all entries (0.016% = 5,217 outliers / 32,404,859 all records)
df_no_outliers = ords_prods_merge[ords_prods_merge['prices'] <= 25]
df_no_outliers.shape

(32399732, 20)

In [18]:
# redo the grouping and aggregate statistics for the subset with no outliers
df_no_outliers.groupby('loyalty_flag').agg({'prices' : ['min', 'median', 'mean', 'max']})

prices                       
                    min median      mean   max
loyalty_flag                                  
Loyal customer      1.0    7.4  7.773575  25.0
New customer        1.0    7.4  7.801206  25.0
Regular customer    1.0    7.4  7.798262  25.0

#### Another way to fix the outliers is to inputting typical values for 2% fat milk and for 2% low fat yogurt
* after a search in the internet I have determined the typical value for a gallon of ```2 % Reduced Fat  Milk``` is 4.99
* similarly, a typical price for a 2lb ```Lowfat 2% Milkfat Cottage Cheese``` is 4.49
* I will assign those values to the data set

In [19]:
ords_prods_merge.loc[ords_prods_merge['product_name'] == r'Lowfat 2% Milkfat Cottage Cheese','prices'] = 4.49

In [20]:
ords_prods_merge.loc[ords_prods_merge['product_name'] == r'2 % Reduced Fat  Milk','prices'] = 4.99

## Step 6 >	The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:
* If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
* If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”


In [21]:
# using the data set without outliers 
# having the outlier prices will increase the price means
result = ords_prods_merge.groupby('user_id')['prices'].transform(np.average)

In [22]:
# check the number of high spenders
(result >= 10).sum()

119662

In [23]:
# check the number of low spenders
(result < 10).sum()

32285197

In [24]:
# create a list that will become the new spending_flag column
# use the for-loop method revised in a previous exercise

#initialize the list
spend_level=[]  
for mean_price in result:
    if (mean_price < 10):
        spend_level.append('Low spender')   # user's mean price < 10
    elif (mean_price >= 10):
        spend_level.append('High spender')  # user's mean price >= 10
        
len(spend_level)

32404859

In [25]:
# assign the list to a new column in the data frame
ords_prods_merge['spending_flag'] = spend_level

In [26]:
ords_prods_merge['spending_flag'].value_counts()

Low spender     32285197
High spender      119662
Name: spending_flag, dtype: int64

In [27]:
# running a sanity check with a few particular users
# where I manually changed the value of user_id to verify the results: price-mean and spending_flag
ords_prods_merge[(ords_prods_merge['user_id']==3)][['user_id', 'prices', 'spending_flag']]

,user_id,prices,spending_flag
692035,3,6.3,Low spender
692036,3,6.3,Low spender
692037,3,6.3,Low spender
692038,3,6.3,Low spender
692039,3,6.3,Low spender
...,...,...,...
4090884,3,6.3,Low spender
4097085,3,9.3,Low spender
4105409,3,7.0,Low spender
4105410,3,7.0,Low spender


In [28]:
# verifying the columns so far
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'first_order_from_customer', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_label', 'busiest_day', 'Busiest_Days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'spending_flag'],
      dtype='object')

## Step 7 >	In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the ```days_since_prior_order``` column. The criteria for the flag should be as follows:
* If the median of “days_since_prior_order” __is higher than 20__, then the customer should be labeled a “__Non-frequent customer__.”
* If the median is __higher than 10 and lower than or equal to 20__, then the customer should be labeled a “__Regular customer__.”
* If the median is __lower than or equal to 10__, then the customer should be labeled a “__Frequent customer__.”


In [29]:
# group by user and obtain the median #days since prior order for each user
median_days_bet_orders = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform(np.median)

In [30]:
len(median_days_bet_orders)

32404859

In [31]:
# create a list with the new order frequency flag
# using the for-loop method 

order_freq_flag = [] #initialize
for x in median_days_bet_orders:
    if (x > 20):
        order_freq_flag.append('Non-frequent customer')
    elif (x > 10):     #implicitly x <= 20
        order_freq_flag.append('Regular customer')
    else:              #implicitly x <= 10
        order_freq_flag.append('Frequent customer')
        
len(order_freq_flag)

32404859

In [32]:
# assign the results to a new column
ords_prods_merge['order_frequency_flag'] = order_freq_flag

In [33]:
#verify the new column is in the dataset
ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'first_order_from_customer', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_label', 'busiest_day', 'Busiest_Days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'spending_flag',
       'order_frequency_flag'],
      dtype='object')

In [34]:
# verify the data set
ords_prods_merge

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order_from_customer,product_id,add_to_cart_order,reordered,...,department_id,prices,price_label,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,spending_flag,order_frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,7,9.0,Mid-range product,Regular busy,regular busy,Most orders,10,New customer,Low spender,Non-frequent customer
1,2398795,1,2,3,7,15.0,False,196,1,1,...,7,9.0,Mid-range product,Regular busy,slowest days,Average orders,10,New customer,Low spender,Non-frequent customer
2,473747,1,3,3,12,21.0,False,196,1,1,...,7,9.0,Mid-range product,Regular busy,slowest days,Most orders,10,New customer,Low spender,Non-frequent customer
3,2254736,1,4,4,7,29.0,False,196,1,1,...,7,9.0,Mid-range product,Least busy,slowest days,Average orders,10,New customer,Low spender,Non-frequent customer
4,431534,1,5,4,15,28.0,False,196,1,1,...,7,9.0,Mid-range product,Least busy,slowest days,Most orders,10,New customer,Low spender,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,1320836,202557,17,2,15,1.0,False,43553,2,1,...,7,3.7,Low-range product,Regular busy,regular busy,Most orders,31,Regular customer,Low spender,Frequent customer
32404855,31526,202557,18,5,11,3.0,False,43553,2,1,...,7,3.7,Low-range product,Regular busy,regular busy,Most orders,31,Regular customer,Low spender,Frequent customer
32404856,758936,203436,1,2,7,NaN,True,42338,4,0,...,19,6.9,Mid-range product,Regular busy,regular busy,Average orders,3,New customer,Low spender,Regular customer
32404857,2745165,203436,2,3,5,15.0,False,42338,16,1,...,19,6.9,Mid-range product,Regular busy,slowest days,Fewest orders,3,New customer,Low spender,Regular customer


## Step 9 >	Export your dataframe as a pickle file and store it correctly in your ```Prepared Data``` folder.

In [35]:
#export in pickle format 
ords_prods_merge.to_pickle(os.path.join(ICpath, '02 Data','Prepared Data','orders_products_aggregated.pkl'))

In [36]:
# fyi only - to track performance of this notebook 
print(' time to run this notebook h:mm:ss: ', datetime.now() - start)

 time to run this notebook h:mm:ss:  0:01:45.525427
